In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../python/data/lfb_jan2013-mar2016 original.csv',sep=",")
data

c:\Users\Thang\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,IncidentNumber,DateOfCall,TimeOfCall,IncidentGroup,StopCodeDescription,SpecialServiceType,PropertyCategory,PropertyType,AddressQualifier,Postcode_full,...,Easting_rounded,Northing_rounded,FRS,IncidentStationGround,FirstPumpArriving_AttendanceTime,FirstPumpArriving_DeployedFromStation,SecondPumpArriving_AttendanceTime,SecondPumpArriving_DeployedFromStation,NumStationsWithPumpsAttending,NumPumpsAttending
0,1131,1.I.13,0:02:06,False Alarm,AFA,NaN,Other Residential,Boarding House/B&B for homeless/asylum seekers,Correct incident address,NW6 1PG,...,525450,184850,London,West Hampstead,167,West Hampstead,NaN,NaN,1,1
1,4131,1.I.13,0:02:09,False Alarm,AFA,NaN,Non Residential,Single shop,Correct incident address,UB6 0HY,...,515450,185450,London,Northolt,236,Northolt,NaN,NaN,1,1
2,5131,1.I.13,0:02:54,False Alarm,AFA,NaN,Non Residential,Other cultural venue,Correct incident address,W6 0RF,...,522450,178650,London,Hammersmith,218,Hammersmith,NaN,NaN,1,1
3,2131,1.I.13,0:03:02,Fire,Secondary Fire,NaN,Outdoor Structure,Small refuse/rubbish container,In street close to,W1H 7DX,...,527850,181050,London,Soho,426,Knightsbridge,NaN,NaN,1,1
4,3131,1.I.13,0:03:03,False Alarm,AFA,NaN,Non Residential,Purpose built office,Within same building,EC3R 7QQ,...,533350,180750,London,Whitechapel,346,Dowgate,648,Whitechapel,2,2
5,6131,1.I.13,0:05:16,Fire,Secondary Fire,NaN,Outdoor,Loose refuse,Open land/water - nearest address to access,NW10 0DQ,...,520750,186650,London,Willesden,323,Willesden,NaN,NaN,1,1
6,7131,1.I.13,0:05:24,False Alarm,AFA,NaN,Non Residential,Church/Chapel,Correct incident address,NW6 7HY,...,525050,184050,London,West Hampstead,448,Belsize,NaN,NaN,1,1
7,10131,1.I.13,0:08:18,False Alarm,False alarm - Good intent,NaN,Dwelling,Purpose Built Flats/Maisonettes - Up to 3 stor...,Within same building,NaN,...,527450,173250,London,Tooting,150,Tooting,152,Tooting,2,3
8,13131,1.I.13,0:12:16,Fire,Primary Fire,NaN,Dwelling,House - single occupancy,Correct incident address,NaN,...,540650,185750,London,Stratford,222,Stratford,260,Stratford,1,2
9,15131,1.I.13,0:16:39,False Alarm,AFA,NaN,Other Residential,Hotel/motel,Correct incident address,UB3 5BP,...,508650,176950,London,Heathrow,274,Heathrow,NaN,NaN,1,1


In [143]:
#no null values in easting and northing
cleanedData = data[data["Easting_m"].notnull()][["Easting_m","Northing_m","IncidentGroup","TimeOfCall"]]

#convert easting northing to lat/lon
#might need to install BNG, -> pip install utm
import pyproj

bng = pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

# AL1 1AB - pyproj.transform(from,to,easting,northing)
lon,lat = pyproj.transform(bng,wgs84, cleanedData["Easting_m"].values, cleanedData["Northing_m"].values)
cleanedData["lat"] = lat;
cleanedData["lon"] = lon;
#cleanedData

#adding hours of call to cleandedData
lst = cleanedData["TimeOfCall"].str.split(':').tolist()
timeCall = [np.int_(item[0]) for item in lst]
cleanedData["HourOfCall"] = timeCall

In [47]:
import json
import numpy as np

from sklearn.cluster import MiniBatchKMeans

#create new dataframe for output dataset
df = pd.DataFrame(data = cleanedData, columns=["lat","lon"])

input = df.values.tolist()
output = input

for i in range(2,7):
    print "k-" + str(i)
    mbk = MiniBatchKMeans(init='k-means++', n_clusters=i, batch_size=45,
                          n_init=10, max_no_improvement=10, verbose=0)
    mbk.fit(input)
    mbk_means_labels = mbk.labels_
    mbk_means_cluster_centers = mbk.cluster_centers_
    mbk_means_labels_unique = np.unique(mbk_means_labels)

    
    print mbk_means_cluster_centers

    cleanedData["kmeans"+str(i)] = mbk_means_labels
    

#for every k-# of a point there is a label(code) which indicates the cluster the point is in
#the code is used for visualization in D3 
df = pd.DataFrame(data = cleanedData, columns=["lat","lon","kmeans2","kmeans3","kmeans4","kmeans5","kmeans6"])

#export to json
df[::3].reset_index().to_json("cluster.json",orient='records')

k-2
[[  5.14988119e+01  -2.79139605e-01]
 [  5.15094002e+01  -4.02149503e-02]]
k-3
[[ 51.50551864  -0.11829686]
 [ 51.51118076  -0.35562883]
 [ 51.51517723   0.08816218]]
k-4
[[ 51.49594442  -0.17385745]
 [ 51.5086977    0.11172957]
 [ 51.51517077  -0.06225949]
 [ 51.50635049  -0.36965009]]
k-5
[[  5.15404821e+01   1.15502169e-01]
 [  5.14336954e+01  -1.73381276e-01]
 [  5.15426238e+01  -1.10882926e-01]
 [  5.14482143e+01   2.32863037e-03]
 [  5.15172934e+01  -3.53647672e-01]]
k-6
[[ 51.54381876  -0.08661409]
 [ 51.50029165  -0.20021947]
 [ 51.51459436  -0.38743269]
 [ 51.55816229   0.19001293]
 [ 51.3955831   -0.12177639]
 [ 51.48589303   0.05500602]]


[11 11]


In [144]:
#Training vars
dataSize = len(cleanedData)
learnSizeKoef = np.int_(0.8 * dataSize)

#splitting data into learning and testing sets
knnLearnData = cleanedData[["lat","lon","HourOfCall"]][:learnSizeKoef].values.tolist()
knnLearnLabels = cleanedData["IncidentGroup"][:learnSizeKoef].values.tolist()

knnTestData = cleanedData[["lat","lon","HourOfCall"]][learnSizeKoef:dataSize].values.tolist()
knnTestLabels = cleanedData["IncidentGroup"][learnSizeKoef:dataSize].values.tolist()

#clean double brackets
knnLearnLabels = np.ravel(knnLearnLabels)
knnTestLabels = np.ravel(knnTestLabels)

In [148]:
#KNN machine learning part

#Training parameters
X = knnLearnData
#labels
y = knnLearnLabels
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10,weights = 'distance')
neigh.fit(X, y) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='distance')

In [149]:
#KNN test predictions
#print neigh.predict([[51.54381876 , -0.08661409],[51.49594442,  -0.17385745]])
#Mean accuracy (prumerna presnost)
neigh.score(knnTestData,knnTestLabels)


0.53483021725202395